In [2]:
import MDAnalysis
from MDAnalysis.tests.datafiles import PSF, DCD   # test trajectory
import numpy.linalg
import numpy as np
import pickle 

In [3]:
u = MDAnalysis.Universe('../data/2agy_final.psf', '../data/2agy-310k-1atm-prod1.dcd')

In [4]:
print(u)

<Universe with 230161 atoms and 229944 bonds>


In [5]:
print(u.atoms.segments)

<SegmentGroup [<Segment AL1>, <Segment BT1>, <Segment AL2>, <Segment BT2>, <Segment XWAT>, <Segment BWAT>, <Segment ION>]>


In [6]:
act_site1 = u.select_atoms('segid BT1 and (resid 39 or resid 58)')
act_site2 = u.select_atoms('segid BT2 and (resid 39 or resid 58)')

In [7]:
dihe = act_site1.dihedrals
dihe[0][0].index

/Users/robert_arbon/anaconda/lib/python2.7/site-packages/IPython/core/formatters.py:92: DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.
  def _ipython_display_formatter_default(self):
/Users/robert_arbon/anaconda/lib/python2.7/site-packages/IPython/core/formatters.py:98: DeprecationWarning: DisplayFormatter._formatters_default is deprecated: use @default decorator instead.
  def _formatters_default(self):
/Users/robert_arbon/anaconda/lib/python2.7/site-packages/IPython/core/formatters.py:677: DeprecationWarning: PlainTextFormatter._deferred_printers_default is deprecated: use @default decorator instead.
  def _deferred_printers_default(self):
/Users/robert_arbon/anaconda/lib/python2.7/site-packages/IPython/core/formatters.py:669: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):
/Users/robert_arbon/anaconda/lib/p

6067

## Extract Active Site

In [8]:
with MDAnalysis.Writer("traj1-as1.dcd", act_site1.n_atoms) as W:
    for ts in u.trajectory:
        W.write(act_site1)

In [9]:
with MDAnalysis.Writer("act_site.pdb", act_site1.n_atoms) as W:
    W.write(act_site1)

### Get internal coordinates

In [30]:
# Dihedrals, angles and index mapping
dihedrals = []
angles = []
bonds = []
indices = []
lens = []
for res in act_site1.residues:
    min_idx = res.atoms.indices.min()
    max_idx = res.atoms.indices.max()
    indices.extend(res.atoms.indices)
    # Dihedrals
    tmp = []
    for dihe in res.atoms.dihedrals:
        too_small = (np.any(dihe.indices < min_idx))
        too_big = (np.any(dihe.indices > max_idx))
        if not(too_small) and not(too_big):
            tmp.append(list(dihe.indices))
    dihedrals.extend(tmp)
    # Angles
    tmp = []
    for ang in res.atoms.angles:
        too_small = (np.any(ang.indices < min_idx))
        too_big = (np.any(ang.indices > max_idx))
        if not(too_small) and not(too_big):
            tmp.append(list(ang.indices))
    angles.extend(tmp)
    # Bonds
    tmp = []
    for bond in res.atoms.bonds:
        too_small = (np.any(bond.indices < min_idx))
        too_big = (np.any(bond.indices > max_idx))
        if not(too_small) and not(too_big):
            if not(np.any(np.array([x[0] for x in bond.type])=='H')):
                tmp.append(list(bond.indices))
    bonds.extend(tmp)
    
# Generate map
indices = np.sort(np.array(indices))
new_indices = np.arange(len(indices))+1
idx_map = dict(zip(indices, new_indices))

# Apply map to angles and dihedrals
dihedrals = np.array(dihedrals)
func = lambda t: idx_map[t]
vfunc = np.vectorize(func)

dihedrals = vfunc(dihedrals)
angles = vfunc(np.array(angles))
bonds = vfunc(np.array(bonds))
# Want all possible 

 
    

In [31]:
pickle.dump(dihedrals, open('act_site_dihe.p', 'wb'))
pickle.dump(angles, open('act_site_ang.p', 'wb'))
pickle.dump(bonds, open('act_site_bond.p', 'wb'))